In [ ]:
import pandas as pd
import re

In [ ]:
import gc
gc.collect()

0

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file_path1 = "/content/drive/My Drive/source.csv"
file_path2 = "/content/drive/My Drive/100_most_prescribed_drugs.csv"
df1 = pd.read_csv(file_path1)
df2 = pd.read_csv(file_path2)
print(df1.head())
print(df2.head())

   Unnamed: 0  pre_id  natinal_id  sex         age          speciality  \
0           0       0           0    0  1377/11/21    General practice   
1           1       1           1    1  1400/02/31          Pediatrics   
2           2       2           2    0  1340/03/14          Cardiology   
3           3       3           3    1  1334/07/06  Emergency medicine   
4           4       4           4    1  1322/02/21    General practice   

   pres_gen_code                                     pres_full_name  \
0          80000               - 21ST CENTURY HAIR, SKIN & NAILS 50   
1          84181                                            - ADZIR   
2          81680  - ANTIAGING CO Q10 RED® SOFTGEL 60 PACKAGE- 60...   
3          81680  - ANTIAGING CO Q10 RED® SOFTGEL 60 PACKAGE- 60...   
4          80102  - APOVITAL CALCIUM CITRATE+MAGNESIUM,ZINC AND ...   

   number_pres_drugs  IHIO_cost  total_cost  age_days  pre_month  
0               10.0          0      442000      8448        

In [ ]:
df1.sort_values(by="pre_id")[:-1]

,Unnamed: 0,pre_id,natinal_id,sex,age,speciality,pres_gen_code,pres_full_name,number_pres_drugs,IHIO_cost,total_cost,age_days,pre_month
0,0,0,0,0,1377/11/21,General practice,80000,"- 21ST CENTURY HAIR, SKIN & NAILS 50",10.0,0,442000,8448,1
1,1,1,1,1,1400/02/31,Pediatrics,84181,- ADZIR,2.0,0,60000,318,1
129393,129393,1,1,1,1400/02/31,Pediatrics,1371,"VITAMIN A CAPSULE, LIQUID FILLED ORAL 50000 [IU]",6.0,5880,8400,318,1
35561,35561,2,2,0,1340/03/14,Cardiology,1395,Bone Mineral Densitometers Genarator,100.0,84000,225000,22218,1
136044,136044,2,2,0,1340/03/14,Cardiology,6788,ROSUVASTATIN TABLET ORAL 20 MG,100.0,0,630000,22218,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16411896,16411896,3093479,330061,1,1344/01/05,Other,637,HYDROCORTISONE (AS SODIUM SUCCINATE) INJECTION...,1.0,0,0,21178,12
16418919,16418919,3093479,330061,1,1344/01/05,Other,1144,"SODIUM CHLORIDE INJECTION, SOLUTION PARENTERAL...",1.0,0,0,21178,12
16405712,16405712,3093479,330061,1,1344/01/05,Other,5616,"BEVACIZUMAB INJECTION, SOLUTION, CONCENTRATE P...",2.0,0,0,21178,12
16408850,16408850,3093480,1115237,1,1338/02/04,Other,383,DAPSONE TABLET ORAL 100 MG,300.0,0,0,23340,12


In [ ]:
# Convert all names in df2 to strings and drop NaN values
df2['براساس کدژنریک'] = df2['براساس کدژنریک'].astype(str)  # Convert to string
df2 = df2.dropna(subset=['براساس کدژنریک'])  # Drop NaN values

# Convert df1 column to string to avoid type issues
df1['pres_full_name'] = df1['pres_full_name'].astype(str)

In [ ]:
names_to_check = df2['براساس کدژنریک'].tolist()

In [ ]:
pattern = '|'.join(names_to_check)  # Create regex pattern from the names
filtered_df = df1[df1['pres_full_name'].str.contains(pattern, case=False, na=False)]

In [ ]:
filtered_df_B12 = filtered_df[~(filtered_df['pres_full_name'].str.contains(r'(?i).*b12.*', na=False, regex = True, case=False) &
                   ~filtered_df['pres_full_name'].str.contains(r'(?i).*b1.*b12.*', na=False, regex = True, case=False))]
filtered_df_B12_Methyl_final = filtered_df_B12[~(filtered_df_B12['pres_full_name'].str.contains(r'(?i).*methylprednisolone.*', na=False, regex = True, case=False))]


In [ ]:
#in haro pak mikoni 1-2 gig ram khali mishe ama baz ham line baadi crash mikone
del filtered_df_B12
del filtered_df
del names_to_check
del df2

In [ ]:
chunk_size = 50000  # Adjust this based on your memory capacity
file_path = '/content/drive/MyDrive/_drug_interaction_filtered_df_final.csv'

# Use chunks to save the DataFrame
with open(file_path, 'w') as f:
    # Write the header only once
    filtered_df_B12_Methyl_final.iloc[:0].to_csv(f, index=False)

    # Write data in chunks
    for start in range(0, len(filtered_df_B12_Methyl_final), chunk_size):
        filtered_df_B12_Methyl_final.iloc[start:start + chunk_size].to_csv(f, header=False, index=False)

In [ ]:
#in ja crash mikard chon hamasho mikhasti ye ja berizi
#filtered_df_B12_Methyl_final.to_csv()

In [ ]:
#filtered_df_without_duplicates = filtered_df_Methyl.drop_duplicates(subset=["pres_full_name"])
#filtered_df_without_duplicates

In [ ]:
Pres_id = filtered_df_B12_Methyl_final["pre_id"]
filtered_result = df1[df1['pre_id'].isin(Pres_id)]

In [ ]:
df_sorted = filtered_result.sort_values(by="pre_id")

In [ ]:
#rasman 2.5 gig ram khali mikone
del filtered_df_B12_Methyl_final
del filtered_result


In [ ]:
df_sorted_without_duplicate = df_sorted.drop_duplicates(subset=["pre_id", "pres_full_name"])

In [ ]:
del df_sorted

In [ ]:
full_range = set(range(1, 3093482))
existing_numbers = set(df_sorted_without_duplicate['pre_id'].dropna().astype(int))
missing_numbers = full_range - existing_numbers

In [26]:
len(missing_numbers)

562520

In [ ]:
df_sorted_without_duplicate[:-1]

,Unnamed: 0,pre_id,natinal_id,sex,age,speciality,pres_gen_code,pres_full_name,number_pres_drugs,IHIO_cost,total_cost,age_days,pre_month
2,2,2,2,0,1340/03/14,Cardiology,81680,- ANTIAGING CO Q10 RED® SOFTGEL 60 PACKAGE- 60...,60.0,0,60,22218,1
118054,118054,2,2,0,1340/03/14,Cardiology,6788,ROSUVASTATIN TABLET ORAL 20 MG,100.0,0,630000,22218,1
70708,70708,2,2,0,1340/03/14,Cardiology,551,FLUOXETINE CAPSULE ORAL 20 MG,60.0,46200,66000,22218,1
144595,144595,2,2,0,1340/03/14,Cardiology,2475,CARVEDILOL TABLET ORAL 6.25 MG,200.0,168000,240000,22218,1
72312,72312,2,2,0,1340/03/14,Cardiology,571,FUROSEMIDE TABLET ORAL 40 MG,100.0,63000,90000,22218,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16420784,16420784,3093456,1095068,1,1331/03/27,Emergency medicine,10427,VITAMIN B1/VITAMIN B6/VITAMIN B12 INJECTION PA...,4.0,0,0,25842,12
16377995,16377995,3093456,1095068,1,1331/03/27,Emergency medicine,1814,VITAMIN D3 INJECTION PARENTERAL 300000 [IU]/1ML,1.0,59587,78818,25842,12
16399979,16399979,3093456,1095068,1,1331/03/27,Emergency medicine,81201,- OSTEOQUINONE F.C TBLET 5*20,30.0,0,0,25842,12
16405632,16405632,3093477,581567,0,1345/08/08,Other,153,"BETAMETHASONE INJECTION, SUSPENSION, EXTENDED ...",1.0,0,0,20593,12


In [ ]:
df_sorted_without_duplicate.shape

(11664451, 13)

In [ ]:
df_sorted_without_duplicate.sort_values(by="pre_id")[:-1]

In [ ]:
chunk_size = 50000  # Adjust this based on your memory capacity
file_path = '/content/drive/MyDrive/drug_interaction_sorted_without_duplicate_11.6M_final.csv'

# Use chunks to save the DataFrame
with open(file_path, 'w') as f:
    # Write the header only once
    df_sorted_without_duplicate.iloc[:0].to_csv(f, index=False)

    # Write data in chunks
    for start in range(0, len(df_sorted_without_duplicate), chunk_size):
        df_sorted_without_duplicate.iloc[start:start + chunk_size].to_csv(f, header=False, index=False)